# Imports #

In [1]:
%%time

# Import libraries and show its versions
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import sys
import pathlib
from pathlib import Path
import glob
from glob import glob
import math
from math import sqrt, log, prod
import collections
from collections import OrderedDict
import pickle
from pickle import dump
import warnings

try:
    import numpy as np; print('NumPy version: ', np.__version__)
    from numpy import array, asarray, delete, append, concatenate
except:
    !{sys.executable} -m pip install numpy==1.19.5
    import numpy as np; print('NumPy version: ', np.__version__)
    from numpy import array, asarray, delete, append, concatenate

try:
    import pandas as pd; print('Pandas version: ', pd.__version__)
    from pandas import read_csv, DataFrame, concat
except:
    !{sys.executable} -m pip install pandas
    import pandas as pd; print('Pandas version: ', pd.__version__)
    from pandas import read_csv, DataFrame, concat
    
try:
    import matplotlib as mpl; print('MatPlotLib version: ', mpl.__version__)
    from matplotlib import pyplot as plt
except:
    !{sys.executable} -m pip install matplotlib
    import matplotlib as mpl; print('MatPlotLib version: ', mpl.__version__)
    from matplotlib import pyplot as plt
    
try:
    import tensorflow as tf; print('TensorFlow version: ', tf.__version__)
    import tensorflow.keras as keras; print('Keras version: ', keras.__version__)
    from tensorflow.keras import backend as K
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
    from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
except:
    !{sys.executable} -m pip install tensorflow
    import tensorflow as tf; print('TensorFlow version: ', tf.__version__)
    import tensorflow.keras as keras; print('Keras version: ', keras.__version__)
    from tensorflow.keras import backend as K
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
    from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau

# Configurations
np.set_printoptions(linewidth=1000)
pd.set_option('display.max_columns', None)
#warnings.filterwarnings('ignore')

print()

NumPy version:  1.18.5
Pandas version:  1.3.4
MatPlotLib version:  3.3.2
TensorFlow version:  2.4.1
Keras version:  2.4.0

CPU times: user 6.72 s, sys: 7.05 s, total: 13.8 s
Wall time: 8.82 s


# Define functions #

In [2]:
%%time

# Date parser
def custom_date_parser(date):
    try:
        return pd.to_datetime(date, format='%Y-%m-%d %H:%M:%S')
    except ValueError:
        return pd.to_datetime(date, format='%d/%m/%Y %H:%M')

def create_univariate_timeseries_dataset(dataset, look_back):
    dataset = asarray(dataset)
    X, y = [], []
    for i in range(len(dataset) - look_back):
        X.append(dataset[i:(i + look_back), 0])
        y.append(dataset[i + look_back, 0])
    return array(X), array(y)

def create_multivariate_timeseries_dataset(data, columns, n_in=1, n_out=1, dropnan=True):
    n_features = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # Input sequence (t-n, ..., t-2, t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [f'{columns[j]} (t-{i})' for j in range(n_features)]
    # Output sequence (t, t+1, ..., t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [f'{columns[j]} (t)' for j in range(n_features)]
        else:
            names += [f'{columns[j]} (t+{i})' for j in range(n_features)]
    # Concatenate sequences
    agg = concat(cols, axis=1)
    agg.columns = names
    # Drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

def update_fixed_size_array(array, new_value, *, axis=0):
    internal_array = array.copy()
    internal_array = delete(internal_array, 0, axis=1)
    internal_array = append(internal_array, new_value, axis=axis)
    return internal_array

def min_max_scaler(array, *, min_limit=None, max_limit=None, min_range=0, max_range=1):
    min_limit = min_limit if min_limit is not None else min(array)
    max_limit = max_limit if max_limit is not None else max(array)
    return [((x - min_limit) / (max_limit - min_limit)) * (max_range - min_range) + min_range
            if max_limit - min_limit != 0 else float(0)
            for x in array]

def min_max_inverse_scaler(array, *, min_limit=None, max_limit=None, min_range=0, max_range=1):
    min_limit = min_limit if min_limit is not None else min(array)
    max_limit = max_limit if max_limit is not None else max(array)
    if max_limit - min_limit == 0:
        return [float(min_limit) for _ in array]
    else:
        scale = (max_range - min_range) / (max_limit - min_limit)
        return [(x - (min_range - min_limit * scale)) / scale for x in array]

def r2(y_true, y_pred):
    y_pred = list(y_pred)
    y_true = list(y_true)
    n = len(y_pred)
    try:
        x_bar = sum(y_pred) / n
        y_bar = sum(y_true) / n
    except ZeroDivisionError:
        return float('nan')
    try:
        x_std = sqrt(sum([(xi - x_bar) ** 2 for xi in y_pred]) / (n - 1))
        y_std = sqrt(sum([(yi - y_bar) ** 2 for yi in y_true]) / (n - 1))
    except ZeroDivisionError:
        return float('nan')
    try:
        zx = [(xi - x_bar) / x_std for xi in y_pred]
    except ZeroDivisionError:
        return float('nan')
    try:
        zy = [(yi - y_bar) / y_std for yi in y_true]
    except ZeroDivisionError:
        return float('nan')
    try:
        r = sum(zxi * zyi for zxi, zyi in zip(zx, zy)) / (n - 1)
    except ZeroDivisionError:
        return float('nan')
    return r ** 2

def rmse(y_true, y_pred, squared=False):
    y_pred = list(y_pred)
    y_true = list(y_true)
    sum_error = 0.0
    for i in range(len(y_true)):
        prediction_error = y_pred[i] - y_true[i]
        sum_error += (prediction_error ** 2)
    try:
        mean_error = sum_error / float(len(y_true))
    except ZeroDivisionError:
        return float('nan')
    return mean_error if squared else sqrt(mean_error)

def mae(y_true, y_pred):
    y_pred = list(y_pred)
    y_true = list(y_true)
    sum_error = 0.0
    for i in range(len(y_true)):
        sum_error += abs(y_pred[i] - y_true[i])
    try:
        return sum_error / float(len(y_true))
    except ZeroDivisionError:
        return float('nan')

def sem(y_true, y_pred):
    y_pred = list(y_pred)
    y_true = list(y_true)
    abs_err = []
    norm_err = []
    for i in range(len(y_true)):
        abs_err.append(abs(y_pred[i] - y_true[i]))
    for i in range(len(y_true)):
        norm_err.append(abs_err[i] / sum(abs_err) if sum(abs_err) != 0 else 0)
    return float(-sum([x * log(x) if x != 0 else 0 for x in norm_err]))

def aem(y_true, y_pred):
    y_pred = list(y_pred)
    y_true = list(y_true)
    abs_err = []
    norm_err = []
    for i in range(len(y_true)):
        abs_err.append(abs(y_pred[i] - y_true[i]))
    for i in range(len(y_true)):
        norm_err.append(abs_err[i] / sum(abs_err) if sum(abs_err) != 0 else 0)
    return float(prod([2 - x ** x for x in norm_err]))

def aicbic(y_true, y_pred):
    y_pred = list(y_pred)
    y_true = list(y_true)
    abs_err = []
    for i in range(len(y_true)):
        abs_err.append(abs(y_pred[i] - y_true[i]))
    return float(log(sum([x * x for x in abs_err])) if sum([x * x for x in abs_err]) != 0 else '-inf')

def mape(actual, pred):
    #actual, pred = np.array(actual), np.array(pred)
    #return np.mean(np.abs((actual - pred) / (actual if actual.all() > 0 else 1e-10))) * 100
    actual, pred = np.array(actual), np.array(pred)
    denominator = np.where(actual != 0, actual, 1)  # Avoid division by zero
    return np.mean(np.abs((actual - pred) / denominator)) * 100

def cvrmse(actual, pred):
    #return rmse(actual, pred)/((np.mean(actual) if np.mean(actual) > 0 else 1e-10) * 100)
    return rmse(actual, pred) / np.mean(actual) * 100

@tf.autograph.experimental.do_not_convert
def K_mse(y_true, y_pred):
    _y_true = y_true[:, 0]
    _y_pred = y_pred[:, 0]
    return K.mean(K.square(_y_pred - _y_true), axis=-1)

@tf.autograph.experimental.do_not_convert
def K_rmse(y_true, y_pred):
    _y_true = y_true[:, 0]
    _y_pred = y_pred[:, 0]
    return K.sqrt(K.mean(K.square(_y_pred - _y_true), axis=-1))

@tf.autograph.experimental.do_not_convert
def K_entropy(y_true, y_pred):
    return -y_pred * K.log(y_true + 1e-10)

class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.lr = []

    def on_epoch_end(self, epoch, logs={}):
        self.lr.append(K.eval(self.model.optimizer.lr))

def scheduler(epoch, lr):
    # Step decay
    #drop = 0.5
    #epochs_drop = 100
    #return lr if (epoch == 0 or epoch % epochs_drop != 0) else lr*drop

    # Step continuous decay
    #drop = 0.5
    #return lr*drop
    
    # Exponential decay
    drop = 0.1
    return lr * tf.math.exp(-drop)

class ReduceLROnPlateauRestoreBestWeights(ReduceLROnPlateau):
    def on_train_begin(self, logs={}):
        self.best_weights = None
        
    def on_epoch_end(self, epoch, logs=None):
        current = logs.get(self.monitor)
        if current is None:
            #logging.warning('Reduce LR on plateau conditioned on metric `%s` '
            #                'which is not available. Available metrics are: %s',
            #                 self.monitor, ','.join(list(logs.keys())))
            pass
        if self.monitor_op(current - self.min_delta, self.best): # New best
            self.best_weights = self.model.get_weights() 
        if not self.monitor_op(current - self.min_delta, self.best): # Not new best
            if not self.in_cooldown(): # And we're not in cooldown
                if self.wait+1 >= self.patience: # Going to reduce lr
                    # Reset best weights so far
                    #print("Backtracking to best model before reducting LR")
                    self.model.set_weights(self.best_weights)
        super().on_epoch_end(epoch, logs) # Actually reduce LR

print()


CPU times: user 365 µs, sys: 0 ns, total: 365 µs
Wall time: 346 µs


# Define parameters #

In [3]:
%%time

# Global variables
multivariate_variables = {
    'Dataset_example': [
        'Variable_example',
    ],
}
steps_to_forecast = {
    'Dataset_example': [12 * 1, 24 * 1],
}

# Dataset variables
datasets = {}
synth_datasets = {}
synth_smooth_datasets = {}
real_synth_datasets = {}
normalization_parameters = {
    'Variable_example': {
        'min_limit': 0,
        'max_limit': 1.5,
        'min_range': 0,
        'max_range': 1
    },
}
look_back = {
    'Dataset_example': 24 * 1,
}
validation_sizes = {
    'Dataset_example': 24 * 1,
}
test_sizes = {
    'Dataset_example': 24 * 1,
}
days_to_validate = 1
test_type = 'real_synth_shuffled'

# Model variables
cnn_parameters = {
    'model': {
        'filters': 64, 
        'kernel_size': 1,
        'strides': 1,
        'activation': 'tanh'
    },
    'fit': {
        'batch_size': 30 * 24 * 4,
        'epochs': 15000,
        'verbose': 0
    },
    'predict': {
        'batch_size': 30 * 24 * 4,
        'verbose': 0
    },
    'compile': {
        'optimizer': 'adam',
        'loss': 'mse',
        #'metrics': [K_rmse]
    },
    'backend': {
        'lr': 0.003
    },
    #'dropout': {
    #    'rate': 0
    #}
}

# Metric variables
available_metrics = {
    'r2': r2,
    'rmse': rmse,
    'mae': mae,
    'sem': sem,
    'aem': aem,
    'aicbic': aicbic,
    'mape': mape,
    'cvrmse': cvrmse
}

# Output variables
output = {}
synth_output = {}
synth_smooth_output = {}
real_synth_output = {}

print()


CPU times: user 94 µs, sys: 85 µs, total: 179 µs
Wall time: 182 µs


# Read datasets #

In [4]:
%%time

# Iterate over all possible train datasets
for file in glob('./datasets/*.csv'):
    
    # Print a header
    print('-' * 37, Path(file).stem.upper(), '-' * 37)
    
    # Read a dataset
    datasets[Path(file).stem] = read_csv(file, parse_dates={'Fecha_Hora': ['Fecha', 'Hora']}, date_parser=custom_date_parser, index_col='Fecha_Hora')
    
    # Update parameters
    steps_to_forecast[Path(file).stem] = [12 * 1, 24 * 1]
    
    # Update parameters
    look_back[Path(file).stem] = max(steps_to_forecast[Path(file).stem]) * 1
    
    # Update parameters
    validation_sizes[Path(file).stem] = max(steps_to_forecast[Path(file).stem]) * 1
    
    # Update parameters
    test_sizes[Path(file).stem] = max(steps_to_forecast[Path(file).stem]) * 1
    
    # Update parameters
    multivariate_variables[Path(file).stem] = [col for col in datasets[Path(file).stem].columns if col not in ['Fecha', 'Hora', 'Fecha_Hora']]
    
    # Select variables
    datasets[Path(file).stem] = datasets[Path(file).stem][multivariate_variables[Path(file).stem]]

    # Iterpolate datasets
    datasets[Path(file).stem] = datasets[Path(file).stem].interpolate('linear')
    
    # Update parameters
    for col in datasets[Path(file).stem].columns:
        col_min = datasets[Path(file).stem][col].min() * 0.9
        col_max = datasets[Path(file).stem][col].max() * 1.1

        # Si la columna ya existe en normalization_parameters, aplicar condiciones de actualización
        if col in normalization_parameters:
            normalization_parameters[col]['min_limit'] = min(normalization_parameters[col]['min_limit'], col_min)
            normalization_parameters[col]['max_limit'] = max(normalization_parameters[col]['max_limit'], col_max)
        else:
            # Si no existe, añadir la columna con los valores calculados
            normalization_parameters[col] = {
                'min_limit': col_min,
                'max_limit': col_max,
                'min_range': 0,
                'max_range': 1
            }
    
    # Print dataset info
    print(datasets[Path(file).stem].info())
    
    # Synth dataset
    synth_datasets[Path(file).stem] = read_csv(f'./datasets/synth/{Path(file).stem}_synth.csv', sep=';', parse_dates=True, index_col=[0])
    synth_datasets[Path(file).stem] = synth_datasets[Path(file).stem][multivariate_variables[Path(file).stem]]
    print(synth_datasets[Path(file).stem].info())
    
    # Real synth dataset
    real_synth_datasets[Path(file).stem] = concat([synth_datasets[Path(file).stem], datasets[Path(file).stem]])
    print(real_synth_datasets[Path(file).stem].info())

"""
# Order datasets
key_order = {k:v for v,k in enumerate([
    '60_Bilbao',
    '60_Madrid',
])}
ordered_dict = OrderedDict(sorted(datasets.items(), key=lambda i:key_order.get(i[0])))
datasets = ordered_dict
"""

print()

------------------------------------- 60_MADRID -------------------------------------
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 421 entries, 2023-09-13 11:00:00 to 2023-09-30 23:00:00
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   CO          421 non-null    float64
 1   O3          421 non-null    float64
 2   PM10        421 non-null    float64
 3   PM2.5       421 non-null    float64
 4   SO2         421 non-null    float64
 5   SpeedWind   421 non-null    float64
 6   DirecWind   421 non-null    int64  
 7   Temp        421 non-null    float64
 8   Humid       421 non-null    int64  
 9   Precip      421 non-null    float64
 10  Car         421 non-null    int64  
 11  Motorcycle  421 non-null    int64  
 12  Bus         421 non-null    int64  
 13  Truck       421 non-null    int64  
dtypes: float64(8), int64(6)
memory usage: 49.3 KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 87600 entr

# Multivariate CNN #

In [5]:
%%time

# Create real-forecasted dataset
data_keys = [k for k in datasets.keys()]
data_variables = list(set([item for sublist in multivariate_variables.values() for item in sublist]))
data_timeseries = ["12h", "24h"]
data_type = ["Real", "Forecasted"]
max_steps = 0
for sf_val in steps_to_forecast.values():
    for v in sf_val:
        if v > max_steps:
            max_steps = v
data_steps = [x for x in range(max_steps)]
data_df = DataFrame([], 
    index=[f'Day {x}' for x in range(1, days_to_validate+1)], 
    columns=[
        list(concatenate([([i]*len(data_variables)*len(data_timeseries)*len(data_type)*len(data_steps)) for i in data_keys], axis=0)),
        len(data_keys)*list(concatenate([([i]*len(data_timeseries)*len(data_type)*len(data_steps)) for i in data_variables], axis=0)),
        len(data_keys)*len(data_variables)*list(concatenate([([i]*len(data_type)*len(data_steps)) for i in data_timeseries], axis=0)),
        len(data_keys)*len(data_variables)*len(data_timeseries)*list(concatenate([([i]*len(data_steps)) for i in data_type], axis=0)),
        len(data_keys)*len(data_variables)*len(data_timeseries)*len(data_type)*[i for i in data_steps]
    ])

# Create a metrics dataframe
#columns = [c for c in list(datasets[next(iter(datasets))].columns)]
columns = list(set(col for dataset in datasets.values() for col in dataset.columns))
days = [f'Day {x}' for x in range(1, days_to_validate+1)]
timeseries = ["12h", "24h"]
metrics = [x.upper() for x in available_metrics.keys()]
metrics_df = DataFrame([], 
       index=[k.upper() for k in datasets.keys()], 
       columns=[
           list(concatenate([([i]*len(days)*len(timeseries)*len(metrics)) for i in columns], axis=0)),
           len(columns)*list(concatenate([([i]*len(timeseries)*len(metrics)) for i in days], axis=0)),
           len(columns)*len(days)*list(concatenate([([i]*len(metrics)) for i in timeseries], axis=0)),
           len(columns)*len(days)*len(timeseries)*[metric for metric in metrics]
       ])

# Iterate over all possible train datasets
for key, dataset in datasets.items():

    # Print a header
    print('-' * 37, key.upper(), '-' * 37)

    # Dataset normalization
    normalized_dataset = dataset.copy()
    for column in dataset.columns:
        normalized_dataset[column] = min_max_scaler(
            normalized_dataset[column].values,
            min_limit=normalization_parameters[column]['min_limit']
            if 'min_limit' in normalization_parameters[column].keys() else None,
            max_limit=normalization_parameters[column]['max_limit']
            if 'max_limit' in normalization_parameters[column].keys() else None,
            min_range=normalization_parameters[column]['min_range']
            if 'min_range' in normalization_parameters[column].keys() else 0,
            max_range=normalization_parameters[column]['max_range']
            if 'max_range' in normalization_parameters[column].keys() else 1)

    # Dataset vectorization
    reframed_dataset = create_multivariate_timeseries_dataset(normalized_dataset.values, normalized_dataset.columns, look_back[key], 1)
    reframed_dataset = reframed_dataset.values
    n_features = len(normalized_dataset.columns)
    n_obs = look_back[key] * n_features
    X_train, y_train = reframed_dataset[:-validation_sizes[key]-days_to_validate*test_sizes[key], :n_obs], reframed_dataset[:-validation_sizes[key]-days_to_validate*test_sizes[key], -n_features:]
    X_validation, y_validation = reframed_dataset[-validation_sizes[key]-days_to_validate*test_sizes[key]:-days_to_validate*test_sizes[key], :n_obs], reframed_dataset[-validation_sizes[key]-days_to_validate*test_sizes[key]:-days_to_validate*test_sizes[key], -n_features:]
    X_test, y_test = reframed_dataset[-days_to_validate*test_sizes[key]:, :n_obs], reframed_dataset[-days_to_validate*test_sizes[key]:, -n_features:]
    
    # Dataset normalization
    normalized_dataset = globals()[f'synth_datasets'][key].copy()
    for column in dataset.columns:
        normalized_dataset[column] = min_max_scaler(
            normalized_dataset[column].values,
            min_limit=normalization_parameters[column]['min_limit']
            if 'min_limit' in normalization_parameters[column].keys() else None,
            max_limit=normalization_parameters[column]['max_limit']
            if 'max_limit' in normalization_parameters[column].keys() else None,
            min_range=normalization_parameters[column]['min_range']
            if 'min_range' in normalization_parameters[column].keys() else 0,
            max_range=normalization_parameters[column]['max_range']
            if 'max_range' in normalization_parameters[column].keys() else 1)

    # Dataset vectorization
    reframed_dataset = create_multivariate_timeseries_dataset(normalized_dataset.values, normalized_dataset.columns, look_back[key], 1)
    reframed_dataset = reframed_dataset.values
    n_features = len(normalized_dataset.columns)
    n_obs = look_back[key] * n_features
    X_train, y_train = reframed_dataset[:-days_to_validate*validation_sizes[key], :n_obs], reframed_dataset[:-days_to_validate*validation_sizes[key], -n_features:]
    #X_validation, y_validation = reframed_dataset[-days_to_validate*validation_sizes[key]:, :n_obs], reframed_dataset[-days_to_validate*validation_sizes[key]:, -n_features:]
        
    # Create the model
    cnn_model = Sequential()
    cnn_model.add(Conv1D( 
        **cnn_parameters['model'],
        input_shape=(look_back[key], n_features)))
    cnn_model.add(MaxPooling1D(pool_size=cnn_parameters['model']['kernel_size']))
    cnn_model.add(Flatten())
    cnn_model.add(Dense(70, activation='tanh'))
    cnn_model.add(Dense(n_features))
    cnn_model.compile(**cnn_parameters['compile'])
    for name, value in cnn_parameters['backend'].items():
        K.set_value(getattr(cnn_model.optimizer, name), value)

    # Fit the model
    cb_loss_history = LossHistory()
    cb_early_stopping = EarlyStopping(monitor='val_loss', mode='min', min_delta=0, patience=300, restore_best_weights=True)
    cb_reduce_learning_rate_on_plateau = ReduceLROnPlateauRestoreBestWeights(monitor='val_loss', mode='min', factor=0.5, patience=10, min_delta=0, min_lr=0.0001)
    history = cnn_model.fit(
        X_train.reshape(
            X_train.shape[0],
            look_back[key],
            n_features),
        y_train,
        **cnn_parameters['fit'],
        validation_data=(X_validation.reshape(
            X_validation.shape[0],
            look_back[key],
            n_features), y_validation),
        shuffle=False,
        callbacks=[cb_loss_history, cb_early_stopping, cb_reduce_learning_rate_on_plateau])
    history.history['loss_history_learning_rate'] = cb_loss_history.lr
    history.history['early_stopped_epoch'] = cb_early_stopping.stopped_epoch
    
    # Check there is a test type
    if test_type is not None:

        # Dataset normalization
        normalized_dataset = dataset.copy()
        for column in dataset.columns:
            normalized_dataset[column] = min_max_scaler(
                normalized_dataset[column].values,
                min_limit=normalization_parameters[column]['min_limit']
                if 'min_limit' in normalization_parameters[column].keys() else None,
                max_limit=normalization_parameters[column]['max_limit']
                if 'max_limit' in normalization_parameters[column].keys() else None,
                min_range=normalization_parameters[column]['min_range']
                if 'min_range' in normalization_parameters[column].keys() else 0,
                max_range=normalization_parameters[column]['max_range']
                if 'max_range' in normalization_parameters[column].keys() else 1)

        # Dataset vectorization
        reframed_dataset = create_multivariate_timeseries_dataset(normalized_dataset.values, normalized_dataset.columns, look_back[key], 1)
        reframed_dataset = reframed_dataset.values
        n_features = len(normalized_dataset.columns)
        n_obs = look_back[key] * n_features
        X_train, y_train = reframed_dataset[:-days_to_validate*validation_sizes[key], :n_obs], reframed_dataset[:-days_to_validate*validation_sizes[key], -n_features:]
        #X_validation, y_validation = reframed_dataset[-days_to_validate*validation_sizes[key]:, :n_obs], reframed_dataset[-days_to_validate*validation_sizes[key]:, -n_features:]
        
        # Fit the model
        cb_loss_history = LossHistory()
        cb_early_stopping = EarlyStopping(monitor='val_loss', mode='min', min_delta=0, patience=300, restore_best_weights=True)
        cb_reduce_learning_rate_on_plateau = ReduceLROnPlateauRestoreBestWeights(monitor='val_loss', mode='min', factor=0.5, patience=10, min_delta=0, min_lr=0.0001)
        history = cnn_model.fit(
            X_train.reshape(
                X_train.shape[0],
                look_back[key],
                n_features),
            y_train,
            **cnn_parameters['fit'],
            validation_data=(X_validation.reshape(
                X_validation.shape[0],
                look_back[key],
                n_features), y_validation),
            shuffle=False,
            callbacks=[cb_loss_history, cb_early_stopping, cb_reduce_learning_rate_on_plateau])
        history.history['loss_history_learning_rate'] = cb_loss_history.lr
        history.history['early_stopped_epoch'] = cb_early_stopping.stopped_epoch

    # Check and create a directory if not exists
    Path('./out/models/').mkdir(parents = True, exist_ok = True)

    # Save the model
    cnn_model.save(f'./out/models/CNN{"" if test_type is None else f"_{test_type}"}_{key}.h5')

    # Check and create a directory if not exists
    Path('./out/history/').mkdir(parents = True, exist_ok = True)

    # Save the history
    with open(f'./out/history/CNN{"" if test_type is None else f"_{test_type}"}_{key}.bin', 'wb') as file:
        dump(history.history, file)

    # Denormalize validation data
    denormalized_y_validation = y_test.copy()
    for index, column in enumerate(dataset.columns):
        denormalized_y_validation[:, index] = min_max_inverse_scaler(
            denormalized_y_validation[:, index],
            min_limit=normalization_parameters[column]['min_limit']
            if 'min_limit' in normalization_parameters[column].keys() else None,
            max_limit=normalization_parameters[column]['max_limit']
            if 'max_limit' in normalization_parameters[column].keys() else None,
            min_range=normalization_parameters[column]['min_range']
            if 'min_range' in normalization_parameters[column].keys() else 0,
            max_range=normalization_parameters[column]['max_range']
            if 'max_range' in normalization_parameters[column].keys() else 1)

    # For each day to forecast
    for day in range(days_to_validate):

        # Print a header
        print('-' * 17, 'Day', str(day+1), '-' * 17)

        # For each time steps to forecast
        for steps in steps_to_forecast[key]:

            # Print a header
            print('-' * 7, str(steps), '-' * 7)

            # Forecast with the model
            cnn_history = asarray(X_test[0]).reshape(1, -1).copy()
            cnn_predictions = asarray([])
            for _ in range(steps):
                _cnn_prediction = cnn_model.predict(
                    cnn_history.reshape(
                        cnn_history.shape[0],
                        look_back[key],
                        n_features),
                    **cnn_parameters['predict'])
                for feature in _cnn_prediction[0]:
                    cnn_history = update_fixed_size_array(cnn_history, [[feature]], axis=1)
                cnn_predictions = append(cnn_predictions, _cnn_prediction[0])
            cnn_predictions = cnn_predictions.reshape(-1, n_features)

            # Denormalize forecast
            denormalized_cnn_predictions = cnn_predictions.copy()
            for index, column in enumerate(dataset.columns):
                denormalized_cnn_predictions[:, index] = min_max_inverse_scaler(
                    denormalized_cnn_predictions[:, index],
                    min_limit=normalization_parameters[column]['min_limit']
                    if 'min_limit' in normalization_parameters[column].keys() else None,
                    max_limit=normalization_parameters[column]['max_limit']
                    if 'max_limit' in normalization_parameters[column].keys() else None,
                    min_range=normalization_parameters[column]['min_range']
                    if 'min_range' in normalization_parameters[column].keys() else 0,
                    max_range=normalization_parameters[column]['max_range']
                    if 'max_range' in normalization_parameters[column].keys() else 1)

            # Store real and forecasted data
            for index, column in enumerate(dataset.columns):
                for step in range(steps):
                    data_df.loc[f'Day {day+1}'][key, column, str(int(int(steps)*int(''.join(filter(str.isdigit, key)))/60))+"h", 'Real', step] = denormalized_y_validation[step][index]
                    data_df.loc[f'Day {day+1}'][key, column, str(int(int(steps)*int(''.join(filter(str.isdigit, key)))/60))+"h", 'Forecasted', step] = denormalized_cnn_predictions[step][index]

            # Print the real data
            #print('REAL DATA:')
            #print(denormalized_y_validation[:steps, :])
            #print()

            # Print the forecasting
            #print('PREDICTIONS:')
            #print(denormalized_cnn_predictions)
            #print()

            # Metrics
            #print('METRICS (NORMALIZED):')
            #for index, column in enumerate(dataset.columns):
            #    print(column.upper())
            #    for mk, mf in available_metrics.items():
            #        print(f'{mk.upper()}: {mf(y_validation[:steps, index], cnn_predictions[:, index])}')
            #    print()
            print('METRICS:')
            for index, column in enumerate(dataset.columns):
                print(column.upper())
                for mk, mf in available_metrics.items():
                    print(f'{mk.upper()}: {mf(denormalized_y_validation[day*steps:day*steps+steps, index], denormalized_cnn_predictions[:, index])}')
                print()

            # Save the metrics
            for index, column in enumerate(dataset.columns):
                for mk, mf in available_metrics.items():
                    metrics_df.loc[key.upper()][column, f'Day {day+1}', str(int(int(steps)*int(''.join(filter(str.isdigit, key)))/60))+"h", mk.upper()] = mf(denormalized_y_validation[day*steps:day*steps+steps, index], denormalized_cnn_predictions[:, index])

            """
            # Check and create a directory if not exists
            Path('./out/metrics/').mkdir(parents = True, exist_ok = True)

            # Save the metrics
            for index, column in enumerate(dataset.columns):
                with open(f'./out/metrics/CNN_{key}_{str(steps)}_{column.upper()}.txt', 'a') as file:
                    for mk, mf in available_metrics.items():
                        file.write(f'{mf(denormalized_y_validation[day*steps:day*steps+steps, index], denormalized_cnn_predictions[:, index])};')
                    file.write(f'\n')
            """

# Check and create a directory if not exists
Path('./out/data/').mkdir(parents = True, exist_ok = True)

# Store the data
data_df.to_csv(f"./out/data/CNN_data{'' if test_type is None else f'_{test_type}'}.csv", sep=";", index=True, decimal=".")

# Check and create a directory if not exists
Path('./out/metrics/').mkdir(parents = True, exist_ok = True)

# Store the metrics
metrics_df.to_csv(f"./out/metrics/CNN_metrics{'' if test_type is None else f'_{test_type}'}.csv", sep=";", index=True, decimal=".")

print()

------------------------------------- 60_MADRID -------------------------------------
----------------- Day 1 -----------------
------- 12 -------
METRICS:
CO
R2: 0.4203427296015642
RMSE: 0.0759679134824911
MAE: 0.06807040940432087
SEM: 2.338979863195945
AEM: 6.815203379769889
AICBIC: -2.6699817879387617
MAPE: 23.265210186834175
CVRMSE: 25.32263782749704

O3
R2: 0.9427425580728084
RMSE: 5.304789404919483
MAE: 4.18693078276184
SEM: 2.171524248031242
AEM: 5.8716203674062895
AICBIC: 5.822126797403145
MAPE: 12.707770850950853
CVRMSE: 15.802434305097348

PM10
R2: 0.6267231801538737
RMSE: 2.756742201913891
MAE: 2.262142943742947
SEM: 2.263506513732966
AEM: 6.431877427490316
AICBIC: 4.513005891644413
MAPE: 13.610217521200676
CVRMSE: 15.781716825635487

PM2.5
R2: 0.6084592423943427
RMSE: 2.0615053764674887
MAE: 1.625383949915569
SEM: 2.178377281173262
AEM: 5.916967436143514
AICBIC: 3.9317796121840707
MAPE: 17.103088261796206
CVRMSE: 21.86790233600872

SO2
R2: nan
RMSE: 0.6371054496660935
MAE: 

<timed exec>:80: RuntimeWarning: invalid value encountered in double_scalars
<timed exec>:152: RuntimeWarning: divide by zero encountered in double_scalars


METRICS:
CO
R2: 0.7804772943008871
RMSE: 0.11297319372658939
MAE: 0.08396880665303778
SEM: 2.8395186742355336
AEM: 11.706577265848537
AICBIC: -1.183155593792615
MAPE: 23.049423302510245
CVRMSE: 32.47133711901971

O3
R2: 0.7670814691741382
RMSE: 25.005516522826774
MAE: 15.833352981238724
SEM: 2.5570430428090423
AEM: 8.782407797700909
AICBIC: 9.616246753226411
MAPE: 33.49514758195778
CVRMSE: 41.30535565571615

PM10
R2: 0.8165051138313115
RMSE: 3.3446394421207954
MAE: 2.7126212959381237
SEM: 2.9080324128763375
AEM: 12.436717016338498
AICBIC: 5.592771625691955
MAPE: 13.053519506961367
CVRMSE: 15.19626483095512

PM2.5
R2: 0.6839514994754952
RMSE: 2.675261044782078
MAE: 2.1785733781804644
SEM: 2.900302670217399
AEM: 12.310718264324153
AICBIC: 5.1461477551901575
MAPE: 18.234721263941122
CVRMSE: 22.538399324184248

SO2
R2: 0.16793106810191658
RMSE: 0.8517662081898904
MAE: 0.6728019386757903
SEM: 2.8570243036750655
AEM: 11.775935626644443
AICBIC: 2.857167443665242
MAPE: 64.64145558819293
CVRMSE